In PySpark there’s no built-in melt() like pandas, but you can reshape (“unpivot”) cleanly with stack() (fast + scalable) or with a dynamic explode trick. Here’s a quick, copy-pasteable guide with a sample DataFrame.

In [0]:
data = [
    (1, "Alice", 10, 20, 30),
    (2, "Bob",   40, 50, None),
    (3, "Cathy", 70, 80, 90),
]
cols = ["id", "name", "math", "science", "english"]
df = spark.createDataFrame(data, cols)
df.display()


id,name,math,science,english
1,Alice,10,20,30
2,Bob,40,50,null
3,Cathy,70,80,90


In [0]:
# Unpivot 3 score columns into two columns: subject, score
unpivot = df.selectExpr(
    "id",
    "name",
    "stack(3, 'math', math, 'science', science, 'english', english) as (subject, score)"
)
unpivot.display()


id,name,subject,score
1,Alice,math,10
1,Alice,science,20
1,Alice,english,30
2,Bob,math,40
2,Bob,science,50
2,Bob,english,null
3,Cathy,math,70
3,Cathy,science,80
3,Cathy,english,90


In [0]:
def melt(df, id_vars, value_vars, var_name="variable", value_name="value"):
    n = len(value_vars)
    pairs = ", ".join([f"'{c}', `{c}`" for c in value_vars])
    return df.selectExpr(
        *id_vars,
        f"stack({n}, {pairs}) as (`{var_name}`, `{value_name}`)"
    )

melted = melt(df, id_vars=["id","name"],
              value_vars=["math","science","english"],
              var_name="subject", value_name="score")
melted.display()


id,name,subject,score
1,Alice,math,10
1,Alice,science,20
1,Alice,english,30
2,Bob,math,40
2,Bob,science,50
2,Bob,english,null
3,Cathy,math,70
3,Cathy,science,80
3,Cathy,english,90


In [0]:
from pyspark.sql.functions import col, lit, array, map_from_arrays, explode

value_vars = [c for c in df.columns if c in ("math","science","english")]  # or any pattern
mapping = map_from_arrays(array([lit(c) for c in value_vars]),
                          array([col(c) for c in value_vars]))

melt_dynamic = df.select("id","name", explode(mapping).alias("subject","score"))
melt_dynamic.display()


id,name,subject,score
1,Alice,math,10
1,Alice,science,20
1,Alice,english,30
2,Bob,math,40
2,Bob,science,50
2,Bob,english,null
3,Cathy,math,70
3,Cathy,science,80
3,Cathy,english,90


In [0]:
data2 = [
    (1, "Alice", 100, 10.0, 120, 12.5, 140, 15.0),  # sales_Q1, profit_Q1, sales_Q2, profit_Q2, ...
    (2, "Bob",    90,  9.5, 110, 11.0, 130, 13.0),
]
cols2 = ["id","name","sales_Q1","profit_Q1","sales_Q2","profit_Q2","sales_Q3","profit_Q3"]
df2 = spark.createDataFrame(data2, cols2)

multi = df2.selectExpr(
    "id","name",
    """
    stack(
      3,
      'Q1', sales_Q1, profit_Q1,
      'Q2', sales_Q2, profit_Q2,
      'Q3', sales_Q3, profit_Q3
    ) as (quarter, sales, profit)
    """
)
multi.display()


id,name,quarter,sales,profit
1,Alice,Q1,100,10.0
1,Alice,Q2,120,12.5
1,Alice,Q3,140,15.0
2,Bob,Q1,90,9.5
2,Bob,Q2,110,11.0
2,Bob,Q3,130,13.0


### Tips & gotchas

Prefer stack() for performance and clarity.

All stacked columns should be compatible types (cast if mixing ints/floats/strings).

To keep all original rows even when all value columns are null, stack() already does the right thing.

Avoid building huge union chains—they’re slow and cluttered.

Want me to tailor the helper to “all columns except id/name”, or to output sorted by id, subject?